In [2]:
# %pip install SpeechRecognition
# %pip install google-api-python-client
# %pip install --upgrade google-auth
# %pip install --upgrade google-cloud-storage
# %pip install --upgrade google-cloud-speech
# %pip install pydub
# %pip install soundfile
# %pip install wave
# %pip install numpy

In [11]:
import os
import time
#from google.cloud import speech_v1p1beta1 as speech
#from google.cloud.speech_v1p1beta1 import enums
from gcp_stt_util import get_filename, save_url_to_gcp_bucket, speech_to_text_beta

In [4]:

from google.cloud import storage
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

### Populate the Google Cloud Storage Credentials from the local credentials .json file

In [5]:
os.environ[
        "GOOGLE_APPLICATION_CREDENTIALS"
    ] = "C:\Dev\LG2 Solutions\Speech-To-Text-Google\gcloud_do_not_share\dictation-sample-82bc323451d9.json"

**Create the audio file dictionary**

In [6]:
audio = dict(
    uri="gs://stt_temp_lg2/Audacity_Test_1.mp3"
)

In [7]:
os.environ["GOOGLE_PROJECT_ID"] = "dictation-sample"
credentials_file = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
credentials = service_account.Credentials.from_service_account_file(
    credentials_file,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [8]:
def create_bucket_if_not_exists(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    if not bucket.exists():
        bucket.create()
        print('Bucket {} created.'.format(bucket_name))
    else:
        print('Bucket {} already exists.'.format(bucket_name))
    return bucket

def upload_file_to_bucket(bucket, from_file_path, file_name):
    blob = bucket.blob(file_name)
    blob.upload_from_filename(from_file_path)
    print('File {} uploaded to bucket.'.format(file_name))

def delete_file_from_bucket(bucket, file_name):
    blob = bucket.blob(file_name)
    blob.delete()
    print('File {} deleted from bucket.'.format(file_name))

def get_text_from_audio_file():
    print('Performing STT')
    time.sleep(5)
    print('STT completed successfully')
    

In [9]:
from pydub import AudioSegment
import wave
import soundfile as sf


def convert_mp3_to_flac(mp3_file):
    sound = AudioSegment.from_file(mp3_file, format='mp3')
    file_dir, file_name = os.path.split(mp3_file)
    flac_file = os.path.join(file_dir, os.path.splitext(file_name)[0] + '.flac')
    sound.export(flac_file, format='flac')
    file_dir, flac_file = os.path.split(flac_file)
    print('File {} converted to {} format.'.format(mp3_file, flac_file))
    return flac_file

def get_sample_rate_hertz(media_file):
    if media_file.endswith('flac'):         
        data, sample_rate = sf.read(media_file)    
    elif media_file.endswith('.wav'): 
        with wave.open(media_file, 'rb') as f:
            sample_rate = f.getframerate()
    return sample_rate


**Create the bucket, upload the audio file**

In [12]:
BUCKET_NAME = 'lg2_solutions_echo_media'
FILE_PATH = './data/Audacity_Test_1.mp3'

bucket = create_bucket_if_not_exists(BUCKET_NAME)
file_dir, file_name = os.path.split(FILE_PATH)
if file_name[-3:] == "mp3":
    file_name = convert_mp3_to_flac(FILE_PATH)
    FILE_PATH = os.path.join(file_dir, file_name)
upload_file_to_bucket(bucket, FILE_PATH, file_name)

Bucket lg2_solutions_echo_media already exists.
File ./data/Audacity_Test_1.mp3 converted to Audacity_Test_1.flac format.
File Audacity_Test_1.flac uploaded to bucket.


## GET THE STT DATA ##

In [13]:
#get_text_from_audio_file()
audio_to_text = dict(uri=f"gs://{BUCKET_NAME}/{file_name}")
sample_rate = get_sample_rate_hertz(FILE_PATH)
#speech_to_text(audio_to_text["uri"],sample_rate)
speech_to_text_beta(audio_to_text["uri"])


ImportError: cannot import name 'enums' from 'google.cloud.speech_v1p1beta1' (c:\Dev\LG2 Solutions\Speech-To-Text-Google\.venv\Lib\site-packages\google\cloud\speech_v1p1beta1\__init__.py)

In [10]:
delete_file_from_bucket(bucket, file_name)

NameError: name 'bucket' is not defined

**Delete the file from the bucket**

**Create or find the bucket**

In [ ]:
file_path = os.path.dirname('./data/Audacity_Test_1.mp3')
bucket_name = 'lg2_solutions_echo_media'
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
if bucket.exists():
    bucket = storage_client.get_bucket(bucket_name)
    print(f"Bucket found: {bucket_name}")
else:
    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "STANDARD"
    new_bucket = storage_client.create_bucket(bucket, location="us")
    print(
        f"Created bucket {new_bucket.name} in {new_bucket.location} with storage class {new_bucket.storage_class}"
    )

**Save the file in the bucket**

# save the ./data/Audacity_Test_1.mp3 file into the bucket
with open(file_path, 'rb') as f:
    

